In [2]:
import pandas as pd

In [3]:
tariff_broadband_df = pd.read_excel(r'Тарифы.xlsx', names=['account', 'connected', 'tariff', 'IP', 'lease'])
payment_broadband_df = pd.read_excel(r'Оплата.xlsx', names=['account', 'datetime', 'amount', 'method']);
CC_call_df = pd.read_excel(r'Обращения в КЦ.xlsx', names=['date', 'time', 'duration', 'account', 'region', 'theme', 'type'])
change_tariff_broadband_df = pd.read_excel(r'Изменения тарифов.xlsx', names=['account', 'datetime', 'new_tariff', 'old_tariff']);

tariff_broadband_df['connected'] = tariff_broadband_df['connected'].astype('datetime64').dt.date
payment_broadband_df['datetime'] = payment_broadband_df['datetime'].astype('datetime64').dt.date
CC_call_df['date'] = CC_call_df['date'].astype('datetime64').dt.date
change_tariff_broadband_df['datetime'] = change_tariff_broadband_df['datetime'].astype('datetime64').dt.date

<h4>1. Укажите три самых частых тарифа</h4>

In [10]:
#находим число пользователей каждого тарифа
topOfTariffByAccounts = tariff_broadband_df['tariff'].value_counts()

#выводим 3 самых популярных
for i, tariff in enumerate(topOfTariffByAccounts.index.tolist()[:3]):
    print('{} место - тариф: "{}"'.format(i + 1, tariff))

1 место - тариф: "План А"
2 место - тариф: "Средний план"
3 место - тариф: "Камбэк 250"


<h4>2. Укажите месяц с самым большим количеством подключений</h4>

In [5]:
#считаем сколько подключений было в каждом месяце
months = {1:'Январь', 2:'Февраль', 3:'Март', 4:'Апрель', 5:'Май', 6:'Июнь', 7:'Июль', 8:'Август', 9:'Сентябрь', 10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'}
monthHasConnections = {}
for mouthIndex in range(1, 13):
    if mouthIndex != 12:
        mask = (tariff_broadband_df['connected'] >= pd.datetime(2018, mouthIndex, 1).date()) & (tariff_broadband_df['connected'] < pd.datetime(2018, mouthIndex + 1, 1).date())
    else:
        mask = (tariff_broadband_df['connected'] >= pd.datetime(2018, 12, 1).date()) & (tariff_broadband_df['connected'] <= pd.datetime(2018, 12, 31).date())
    monthHasConnections.update({months[mouthIndex]: tariff_broadband_df.loc[mask, 'connected'].count()})

#выводим месяц с наибольшим числом подключений
monthWithMaxConnections = max(monthHasConnections, key=lambda key: monthHasConnections[key])
print('месяц с наибольшим числом подключений: {}'.format(monthWithMaxConnections))

месяц с наибольшим числом подключений: Октябрь


<h4>3. Найдите корреляцию между наличием у абонента статического IP адреса и наличием роутера в аренде. Что можно сказать о связи друг с другом этих свойств?</h4>

In [90]:
corr = tariff_broadband_df['IP'].corr(tariff_broadband_df['lease'], method='spearman')
print('корреляция = {}'.format(corr))
print('вывод - корреляция практически нулевая, значит, что эти величины не зависят друг от друга. Отрицательное заничение означает бы, что чаще IP и роутер - это взаимоисключающие услуги')

корреляция = -0.040385354255088275
вывод - корреляция практически нулевая, значит, что эти величины не зависят друг от друга. Отрицательное заничение означает бы, что чаще IP и роутер - это взаимоисключающие услуги


<h4>4. Определите среднее, моду, медиану, 1 и 3 квартили (Q1 и Q3) поступивших платежей</h4>

In [91]:
#считаем различные параметры:
mean = payment_broadband_df.amount.describe()['mean']
mode = payment_broadband_df.amount.mode()[0]
median = payment_broadband_df.amount.median()
Q1 = payment_broadband_df.amount.describe()['25%']
Q3 = payment_broadband_df.amount.describe()['75%']

print('среднее = {}'.format(mean))
print('мода = {}'.format(mode))
print('медиана = {}'.format(median))
print('Q1 = {}'.format(Q1))
print('Q3 = {}'.format(Q3))

среднее = 481.5872599645292
мода = 500.0
медиана = 500.0
Q1 = 250.0
Q3 = 590.0


<h4>5. Определите месяц с наибольшей суммой платежей</h4>

In [92]:
#считаем суммы платежей для каждого месяца
months = {1:'Январь', 2:'Февраль', 3:'Март', 4:'Апрель', 5:'Май', 6:'Июнь', 7:'Июль', 8:'Август', 9:'Сентябрь', 10:'Октябрь', 11:'Ноябрь', 12:'Декабрь'}
monthHasPayments = {}
for mouthIndex in range(1, 13):
    if mouthIndex != 12:
        mask = (payment_broadband_df['datetime'] >= pd.datetime(2018, mouthIndex, 1).date()) & (payment_broadband_df['datetime'] < pd.datetime(2018, mouthIndex + 1, 1).date())
    else:
        mask = (payment_broadband_df['datetime'] >= pd.datetime(2018, 12, 1).date()) & (payment_broadband_df['datetime'] <= pd.datetime(2018, 12, 31).date())
    monthHasPayments.update({months[mouthIndex]: payment_broadband_df.loc[mask, 'amount'].sum(axis=0)})

#выводим месяц с наибольшей суммой платежей
monthWithMaxPayments = max(monthHasPayments, key=lambda key: monthHasPayments[key])
print('месяц с наибольшей суммой платежей: {}'.format(monthWithMaxPayments))

месяц с наибольшей суммой платежей: Декабрь


<h4>6. Найдите 3 договора, заплативших наибольшую сумму</h4>

In [93]:
#если необходимо смотреть сумму единоразового платежа по договору

#выводим три строки с наибольшим значением amount:
top3accountsByAmount = payment_broadband_df.nlargest(3, 'amount').account.tolist()
for i in range(len(top3accountsByAmount)):
    print('{} место - договор: {}'.format(i + 1, top3accountsByAmount[i]))

1 место - договор: 100667
2 место - договор: 781442
3 место - договор: 184287


In [94]:
#если необходимо смотреть сумму всех платежей по договору

#находим список всех договоров
accounts = payment_broadband_df.account.value_counts().index.tolist()

#находим, сколько суммарно было заплочено каждым договором
accountHasSumOfAmounts = {}
for account in accounts:
    sumOfAmount =  payment_broadband_df[payment_broadband_df['account'] == account].amount.sum(axis=0)
    accountHasSumOfAmounts.update({account: sumOfAmount})

#сортируем по возрастанию и выводим 3 заплативших самую большую сумму
arraySortedAccounts = list(accountHasSumOfAmounts.items())
arraySortedAccounts.sort(key=lambda i: i[1], reverse = True)
for key, val in arraySortedAccounts[:3]:
    print('договор "{}" - было заплочено: {}'.format(key, val))

договор "781442" - было заплочено: 29000.0
договор "1532729" - было заплочено: 20000.0
договор "1219811" - было заплочено: 19516.0


<h4>7. Найдите самый частый метод оплаты у клиентов, подключившихся в октябре</h4>

In [95]:
#находим список всех оплат в октябре
mask = (payment_broadband_df['datetime'] >= pd.datetime(2018, 10, 1).date()) & (payment_broadband_df['datetime'] <= pd.datetime(2018, 11, 1).date())
peymentsInOctober = payment_broadband_df.loc[mask, 'method']

#находим наиболее часто встречающийся
print('наиболее часто встречающийся тип оплаты: {}'.format(peymentsInOctober.mode()[0]))

наиболее часто встречающийся тип оплаты: Альфа


<h4>8. Найдите 3 договора, на которых наиболее часто меняли тарифы</h4>

In [96]:
#находим сколько раз упоминается каждый договор
accountsFreq = change_tariff_broadband_df.account.value_counts()

#находим топ-3
top3accounts = accountsFreq.index.tolist()[:3]
for i in range(len(top3accounts)):
    print('{} метсто - договор: {}'.format(i + 1, top3accounts[i]))

1 метсто - договор: 105594
2 метсто - договор: 157175
3 метсто - договор: 298677


<h4>9. Укажите тариф, с которого "уходили" (переключались на другой) чащей всего</h4>

In [97]:
#выводим наиболее часто встречающийся тариф в столбце "old_tariff" 
tariff = change_tariff_broadband_df.old_tariff.mode().tolist()[0]
print('тариф с которого уходили чаще всего: {}'.format(tariff))

тариф с которого уходили чаще всего: План А


<h4>10. Укажите договор клиента, который подключившись в 2018 году, обращался в КЦ чаще всего</h4>

In [98]:
#составляем список всех пользователей за 2018 год
accountsCreatedIn2018 = tariff_broadband_df.account.tolist()

#составляем ТОП пользователей по числу обращений
topChatFrequencyUsers = CC_call_df.account.value_counts()

#проходим по ТОПу в поисках аккаунта, зарегестрированного в 2018 году
for account in topChatFrequencyUsers.index.tolist():
    if account in accountsCreatedIn2018:
        contactedMostOften = account
        break
        
print('клиент, зарегестрировавшийся в 2018 и обращавшийся чаще всего: {}'.format(contactedMostOften))

клиент, зарегестрировавшийся в 2018 и обращавшийся чаще всего: 244516


<h4>11. Укажите шестичасовой интервал, в котором было больше всего обращений в КЦ</h4>

In [86]:
#если речь идет только о четырех промежутках 0:00-06:00, 06:00-12:00, 12:00-18:00, 18:00-00:00 
#(стартовый момент времнеи передвигаем каждый раз на 6 часов)

#сколько-часовой промежуток
deltaTime = 6

#объект для хранения времени старта для интервала с наибольшим числом обращений
timeIntervalWithMaxRequest = {'h': 0, 'm': 0, 'countOfRequests': 0} 

#перебираем все временные промежутки с шагом в 1 минуту
for h in range(0, 24, deltaTime):
    timeMomentStart = h * 3600
    timeMomentEnd = timeMomentStart + deltaTime * 3600
    mask = ((pd.to_datetime(CC_call_df['time'],format= '%H:%M:%S' ).dt.time > pd.to_datetime(timeMomentStart, unit='s').time()) & 
            (pd.to_datetime(CC_call_df['time'],format= '%H:%M:%S' ).dt.time < pd.to_datetime(timeMomentEnd, unit='s').time()))
    countOfRequests = CC_call_df.loc[mask].count().time

    #обновляем объект, если число обращений больше прежнего 
    if timeIntervalWithMaxRequest['countOfRequests'] < countOfRequests:
        timeIntervalWithMaxRequest = {'h': h, 'm': '00', 'countOfRequests': countOfRequests}

print('временной промежуток с наибольшим числом запросов:')
print('с {}:{} до {}:{} было {} запроса'.format(timeIntervalWithMaxRequest['h'], 
                                                    timeIntervalWithMaxRequest['m'], 
                                                    timeIntervalWithMaxRequest['h'] + deltaTime, 
                                                    timeIntervalWithMaxRequest['m'], 
                                                    timeIntervalWithMaxRequest['countOfRequests']))

временной промежуток с наибольшим числом запросов:
с 12:00 до 18:00 было 40481 запроса


In [87]:
%%time
#если речь идет только о 24*60=1440 промежутках
#(стартовый момент времнеи передвигаем каждый раз на 1 минуту)

#сколько-часовой промежуток
deltaTime = 6

#объект для хранения времени старта для интервала с наибольшим числом обращений
timeIntervalWithMaxRequest = {'h': 0, 'm': 0, 'countOfRequests': 0} 

#перебираем все временные промежутки с шагом в 1 минуту
for h in range(24 - deltaTime):
    for m in range(60):
        timeMomentStart = h * 3600 + m * 60
        timeMomentEnd = timeMomentStart + deltaTime * 3600
        mask = ((pd.to_datetime(CC_call_df['time'],format= '%H:%M:%S' ).dt.time > pd.to_datetime(timeMomentStart, unit='s').time()) & 
                (pd.to_datetime(CC_call_df['time'],format= '%H:%M:%S' ).dt.time < pd.to_datetime(timeMomentEnd, unit='s').time()))
        countOfRequests = CC_call_df.loc[mask].count().time
        
        #обновляем объект, если число обращений больше прежнего 
        if timeIntervalWithMaxRequest['countOfRequests'] < countOfRequests:
            timeIntervalWithMaxRequest = {'h': h, 'm': m, 'countOfRequests': countOfRequests}

print('временной промежуток с наибольшим числом запросов:')
print('с {}:{} до {}:{}  было {} запроса'.format(timeIntervalWithMaxRequest['h'], 
                                                    timeIntervalWithMaxRequest['m'], 
                                                    timeIntervalWithMaxRequest['h'] + deltaTime, 
                                                    timeIntervalWithMaxRequest['m'], 
                                                    timeIntervalWithMaxRequest['countOfRequests']))

временной промежуток с наибольшим числом запросов:
с 10:52 до 16:52  было 40955 запроса
CPU times: user 6min 23s, sys: 1.19 s, total: 6min 24s
Wall time: 6min 25s


<h4>12. Определите среднее время разговора с КЦ клиентов из г. Обнинск</h4>

In [88]:
#находим продолжительности всех звонков из г. Обнинск
callsHasDuration = CC_call_df[CC_call_df['region'] == 'г. Обнинск'].duration

#находим среднее значение
meanDuration = callsHasDuration.describe()['mean']
print('средняя продолжительность разговора: {}с'.format(meanDuration))

средняя продолжительность разговора: 232.44с


<h4>13. Укажите 5 наиболее часто встречающихся типов обращений в КЦ (в порядке убывания их кол-ва) клиентов, которые подключились в 2018 году и сделали платежей на сумму от 10 000 руб.</h4>

In [89]:
#составляем список всех пользователей за 2018 год
accountsCreatedIn2018 = tariff_broadband_df.account.tolist()

#находим сумму платежей по договору каждого клиента, подключенного в 2018
mask = payment_broadband_df.account.isin(accountsCreatedIn2018)
accountHasPayments = payment_broadband_df.loc[mask, ['account', 'amount']].groupby('account').amount.sum()

#отсееваем договора, в которых сумма менее 10000
mask = accountHasPayments > 10000
accountHasSumOfAmountsOver10k = accountHasPayments[mask].index.tolist()

#находим число обращений по каждому типу обращений среди пользователей подходящих под условия (подкл. в 2018г и сумма платежей >10k)
mask = (CC_call_df.account.isin(accountHasSumOfAmountsOver10k))
typeHasCount = CC_call_df.loc[mask].type.value_counts()

#находим ТОП 5 типов обращений по числу обращений
top5types = typeHasCount.index.tolist()[:5]
for i in range(len(top5types)):
    print('{} место - тип обращения: {}'.format(i + 1, top5types[i]))

1 место - тип обращения: Проблема с Интернет
2 место - тип обращения: Иная консультация
3 место - тип обращения: Общая проблема
4 место - тип обращения: Вопрос по настройке оборудования
5 место - тип обращения: Повторное обращение по техническому ТТ
